In [1]:
%AddDeps org.sameersingh.scalaplot scalaplot 0.0.4
%AddDeps com.google.protobuf protobuf-java 2.6.1
%AddDeps com.databricks spark-csv_2.10 1.5.0 --transitive

Marking org.sameersingh.scalaplot:scalaplot:0.0.4 for download
Preparing to fetch from:
-> file:/tmp/toree_add_deps3630819211546819142/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree_add_deps3630819211546819142/https/repo1.maven.org/maven2/org/sameersingh/scalaplot/scalaplot/0.0.4/scalaplot-0.0.4.jar
Marking com.google.protobuf:protobuf-java:2.6.1 for download
Preparing to fetch from:
-> file:/tmp/toree_add_deps3630819211546819142/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree_add_deps3630819211546819142/https/repo1.maven.org/maven2/com/google/protobuf/protobuf-java/2.6.1/protobuf-java-2.6.1.jar
Marking com.databricks:spark-csv_2.10:1.5.0 for download
Preparing to fetch from:
-> file:/tmp/toree_add_deps3630819211546819142/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree_add_deps3630819211546819142/https/repo1.maven.org/maven2/com/univocity/univocity-parsers/1.5.1/univocity-parsers-1.5.1.jar
-> New file at /tmp/toree_add_deps3630819211546819142/htt

In [2]:
val sqlContext = org.apache.spark.sql.SQLContext.getOrCreate(sc)

In [3]:
import sqlContext.implicits._
import org.apache.spark.sql.functions._

### Preconditions
- Make sure data has been downloaded with `download_data.py`
- Make sure the first pass of cleaning has been performed with `clean_files.scala`

In [16]:
val files = List("cooking", "crypto", "robotics", "biology", "travel", "diy")

/* Load and print all files */
val df_all = files.map(f => {
                        sqlContext.read.format("com.databricks.spark.csv").option("header", "true").load("../dat/"+f+"_clean.csv")
                   }).reduce(_ unionAll _).withColumn("tags", split($"tags", " "))

In [17]:
df_all.show

+---+--------------------+--------------------+--------------------+
| id|               title|             content|                tags|
+---+--------------------+--------------------+--------------------+
|  1|How can I get che...| My chocolate chi...|[baking, cookies,...|
|  2|How should I cook...| I've heard of pe...|[oven, cooking-ti...|
|  3|What is the diffe...| I always use bro...|              [eggs]|
|  4|What is the diffe...| And can I use on...|[substitutions, p...|
|  5|In a tomato sauce...| It seems that ev...|[sauce, pasta, to...|
|  6|What ingredients ...| I have a recipe ...|[substitutions, h...|
|  9|What is the inter...| I'd like to know...|[food-safety, bee...|
| 11|How should I poac...| What's the best ...|[eggs, basics, po...|
| 12|How can I make my...| My ice cream doe...|         [ice-cream]|
| 17|How long and at w...| I'm interested i...|[baking, chicken,...|
| 23|Besides salmon, w...| I've fallen in l...|[grilling, salmon...|
| 27|Do I need to sift...| Is ther

In [18]:
df_all.count

87000

In [27]:
import sqlContext.implicits._
import org.apache.spark.sql.Row
val rdd = df_all.select("tags").rdd.map{case Row(t: Seq[String]) => t}

In [28]:
val tags = rdd.collect.flatMap(_.map(t => (t, t.split("-").size)))
val tagsDF = tags.toSeq.distinct.toDF("tag", "length")

tags.maxBy(_._2)

(meet-in-the-middle-attack,5)

In [29]:
tagsDF.show

+--------------------+------+
|                 tag|length|
+--------------------+------+
|              baking|     1|
|             cookies|     1|
|             texture|     1|
|                oven|     1|
|        cooking-time|     2|
|               bacon|     1|
|                eggs|     1|
|       substitutions|     1|
|please-remove-thi...|     4|
|         baking-soda|     2|
|       baking-powder|     2|
|               sauce|     1|
|               pasta|     1|
|            tomatoes|     1|
|     italian-cuisine|     2|
|               herbs|     1|
|             parsley|     1|
|         food-safety|     2|
|                beef|     1|
|              basics|     1|
+--------------------+------+
only showing top 20 rows



### TODO: load all data and do a histogram over tag lengths

In [30]:
tagsDF.sort(-$"length").show(false)

+-------------------------+------+
|tag                      |length|
+-------------------------+------+
|meet-in-the-middle-attack|5     |
|san-francisco-bay-area   |4     |
|change-purpose-of-travel |4     |
|t-and-t-citizens         |4     |
|90-180-visa-rules        |4     |
|us-visa-waiver-program   |4     |
|proof-of-onward-travel   |4     |
|man-in-the-middle        |4     |
|proof-provenance-of-funds|4     |
|great-wall-of-china      |4     |
|ho-chi-minh-city         |4     |
|please-remove-this-tag   |4     |
|gondolas-and-cable-cars  |4     |
|half-and-half            |3     |
|brute-force-attack       |3     |
|middle-eastern-cuisine   |3     |
|deep-dish-pizza          |3     |
|cut-of-meat              |3     |
|known-plaintext-attack   |3     |
|side-channel-attack      |3     |
+-------------------------+------+
only showing top 20 rows



In [31]:
import org.sameersingh.scalaplot.Implicits._
val data = tags.map(_._2).groupBy(identity).mapValues(_.length)
val (x, y) = data.toSeq.sortBy(_._1).map(x => (x._1.toDouble, x._2.toDouble)).unzip

In [1]:
kernel.magics.html(output(SVG, barChart(x -> y)))

Name: Compile Error
Message: <console>:19: error: not found: value output
              kernel.magics.html(output(SVG, barChart(x -> y)))
                                 ^
StackTrace: 